# **Data Collection**

### Objectives

* Fetch data from Kaggle and save as raw data / and prepare it for further processes
* save data downloaded from Kaggle in the dataset collection directory, inputs/dataset/collection

### Inputs

* Kaggle JSON file - authentication token

### Outputs

* Generate Dataset: inputs/datasets/collection/mildew-dataset 

### Additional Comments

* The client provided the data under an NDA (non-disclosure agreement), therefore the data will only be shared with professionals that are officially involved in the project.

### Insights | Conclusions
* text here if any

---

## Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/mildew-detection/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/mildew-detection'

## Fetch data from Kaggle

Install kaggle to fetch data

In [5]:
# install kaggle package
! pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 37.8 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73031 sha256=f6ada83d1797986a991e9ee6afc87331cdb04071aa0d926703bc304901ce9067
  Stored in directory: /home/gitpod/.cache/pip/wheels/29/da/11/144cc25aebdaeb4931b231e25fd34b394e6a5725cbb2f50106
Successfully built kaggle


---

Change kaggle configuration directory to current working directory and permission of kaggle authentication json so the token is recognized in the session

In [11]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

* Define the dataset path - The path is the text after [https://kaggle.com/datasets/](https://kaggle.com/datasets/)
* Define the destination folder
* Download the dataset from Kaggle

In [12]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/datasets/raw"
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

 89%|█████████████████████████████████▊    | 49.0M/55.0M [00:01<00:00, 46.1MB/s]
100%|██████████████████████████████████████| 55.0M/55.0M [00:01<00:00, 42.8MB/s]


Unzip and delete the downloaded file

In [13]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')

delete kaggle.json file

In [14]:
rm kaggle.json

# Section 2

Section 2 content

---

NOTE

* You may add how many sections you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section for "Conclusions and Next Steps" and state your conclusions and next steps.

In [4]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)


IndentationError: expected an indented block (2852421808.py, line 5)